In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import random 
from datetime import datetime, date, time, timedelta

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE

In [10]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

### Calculate differences between consequtive CGM values

In [17]:
data = pd.read_csv('data/cgm_to_meal_start.csv')
data = data.dropna()
cgm_diff = data['cgm'].diff()
data['diff'] = cgm_diff
data = data.dropna()
data = data.reset_index(drop=True)

#### Calculate average time between CGM samples

In [18]:
cgm_time = data['time_cgm'].values
difference = []
for i in range(len(cgm_time)-1):
    start = datetime.strptime(cgm_time[i], "%Y-%m-%d %H:%M:%S")
    end = datetime.strptime(cgm_time[i+1], "%Y-%m-%d %H:%M:%S")
    x = end-start
    difference.append(x.total_seconds())

difference = np.array(difference)
diff_mean = np.mean(difference)/60
print(diff_mean)

7.531803002847528


#### Get the 10 samples prior to 45 minutes of a meal and a non-meal point

In [19]:
near_param = 10
leaving_numbers = 6 # corresponds to going back 45 minutes

meal_indices = data.index[data['meal_start'] == 1]
meal_data_X = []
for idx in meal_indices:
    near_idx_points = data.iloc[list(range(idx - leaving_numbers - 1, idx - leaving_numbers - near_param - 1, -1))]
    meal_data_X.append(near_idx_points['diff'].to_list())

non_meal_indices = data.index[data['meal_start'] == 0]
non_meal_data_X = []
for idx in non_meal_indices:
    near_idx_points = data.iloc[list(range(idx - leaving_numbers - 1, idx - leaving_numbers - near_param - 1, -1))]
    non_meal_data_X.append(near_idx_points['diff'].to_list())


#### Undersample non-meal points for a more balanced dataset, oversample meal-point, and calculate metric over multiple trails

In [74]:
TOTAL_TRAILS = 100

all_true_positive = []
all_true_negative = []
all_false_positive = []
all_false_negative = []
all_accuracies = []

for trial in range(TOTAL_TRAILS):
    meal_data_Y = np.ones(len(meal_data_X))
    non_meal_data_Y = np.zeros(len(non_meal_data_X))
    meal_data_X = np.array(meal_data_X)
    non_meal_data_X = np.array(non_meal_data_X)

    numPoints_non_meal = 2000

    sampled_indices_non_meal = np.array(random.sample(range(0,non_meal_data_X.shape[0]),numPoints_non_meal))

    trainDataX_non_meal = non_meal_data_X[sampled_indices_non_meal]
    trainDataX_meal = meal_data_X
    trainDataY_non_meal = non_meal_data_Y[sampled_indices_non_meal]
    trainDataY_meal = meal_data_Y

    Data_in = np.concatenate((trainDataX_meal,trainDataX_non_meal),axis=0)
    Data_out = np.concatenate((trainDataY_meal,trainDataY_non_meal),axis=0)

    clf = RandomForestClassifier(max_depth=5, n_estimators=10)

    oversample = SMOTE()

    X, y = Data_in, Data_out
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.3, random_state=RANDOM_SEED)

    X_train, y_train = oversample.fit_resample(X_train, y_train)

    reg = clf.fit(X_train, y_train)
    accuracy = reg.score(X_test, y_test)
    pred = reg.predict(X_test)
    cm = (confusion_matrix(y_test, pred))
    
    true_positive = cm[1][1] / sum(cm[1])
    true_negative = cm[0][0] / sum(cm[0])
    
    false_positive = cm[0][1] / sum(cm[0])
    false_negative = cm[1][0] / sum(cm[1])
    
    print("#" * 10)
    print("Trail ", trial)
    print("TNR (non-meal -> non-meal)", true_negative)
    print("TPR (meal -> meal)", true_positive)
    print("FPR (non-meal -> meal)", false_positive)
    print("FNR (meal -> non-meal)", true_negative)

    print("Accuracy: ", accuracy )
    print("#" * 10)
    print("\n")
    all_true_positive.append(true_positive)
    all_true_negative.append(true_negative)
    all_false_positive.append(false_positive)
    all_false_negative.append(false_negative)
    all_accuracies.append(accuracy)

##########
Trail  0
TNR (non-meal -> non-meal) 0.7092436974789916
TPR (meal -> meal) 0.2616822429906542
FPR (non-meal -> meal) 0.2907563025210084
FNR (meal -> non-meal) 0.7092436974789916
Accuracy:  0.5908529048207664
##########


##########
Trail  1
TNR (non-meal -> non-meal) 0.7058823529411765
TPR (meal -> meal) 0.3644859813084112
FPR (non-meal -> meal) 0.29411764705882354
FNR (meal -> non-meal) 0.7058823529411765
Accuracy:  0.61557478368356
##########


##########
Trail  2
TNR (non-meal -> non-meal) 0.7025210084033613
TPR (meal -> meal) 0.29906542056074764
FPR (non-meal -> meal) 0.29747899159663865
FNR (meal -> non-meal) 0.7025210084033613
Accuracy:  0.595797280593325
##########


##########
Trail  3
TNR (non-meal -> non-meal) 0.7378151260504202
TPR (meal -> meal) 0.24766355140186916
FPR (non-meal -> meal) 0.26218487394957984
FNR (meal -> non-meal) 0.7378151260504202
Accuracy:  0.6081582200247219
##########


##########
Trail  4
TNR (non-meal -> non-meal) 0.7008403361344537
TPR (mea

##########
Trail  37
TNR (non-meal -> non-meal) 0.7478991596638656
TPR (meal -> meal) 0.3130841121495327
FPR (non-meal -> meal) 0.25210084033613445
FNR (meal -> non-meal) 0.7478991596638656
Accuracy:  0.6328800988875154
##########


##########
Trail  38
TNR (non-meal -> non-meal) 0.6756302521008404
TPR (meal -> meal) 0.35514018691588783
FPR (non-meal -> meal) 0.3243697478991597
FNR (meal -> non-meal) 0.6756302521008404
Accuracy:  0.5908529048207664
##########


##########
Trail  39
TNR (non-meal -> non-meal) 0.7310924369747899
TPR (meal -> meal) 0.2803738317757009
FPR (non-meal -> meal) 0.2689075630252101
FNR (meal -> non-meal) 0.7310924369747899
Accuracy:  0.6118665018541409
##########


##########
Trail  40
TNR (non-meal -> non-meal) 0.7310924369747899
TPR (meal -> meal) 0.27102803738317754
FPR (non-meal -> meal) 0.2689075630252101
FNR (meal -> non-meal) 0.7310924369747899
Accuracy:  0.6093943139678616
##########


##########
Trail  41
TNR (non-meal -> non-meal) 0.6722689075630253
TP

TNR (non-meal -> non-meal) 0.7075630252100841
TPR (meal -> meal) 0.2850467289719626
FPR (non-meal -> meal) 0.292436974789916
FNR (meal -> non-meal) 0.7075630252100841
Accuracy:  0.595797280593325
##########


##########
Trail  76
TNR (non-meal -> non-meal) 0.7109243697478992
TPR (meal -> meal) 0.34579439252336447
FPR (non-meal -> meal) 0.28907563025210087
FNR (meal -> non-meal) 0.7109243697478992
Accuracy:  0.6143386897404203
##########


##########
Trail  77
TNR (non-meal -> non-meal) 0.761344537815126
TPR (meal -> meal) 0.2616822429906542
FPR (non-meal -> meal) 0.23865546218487396
FNR (meal -> non-meal) 0.761344537815126
Accuracy:  0.6291718170580964
##########


##########
Trail  78
TNR (non-meal -> non-meal) 0.6840336134453782
TPR (meal -> meal) 0.2616822429906542
FPR (non-meal -> meal) 0.31596638655462184
FNR (meal -> non-meal) 0.6840336134453782
Accuracy:  0.5723114956736712
##########


##########
Trail  79
TNR (non-meal -> non-meal) 0.7361344537815127
TPR (meal -> meal) 0.22897

## Average Metrics

In [75]:
print("TNR (non-meal -> non-meal)", np.mean(true_negative))
print("TPR (meal -> meal)", np.mean(true_positive))
print("FPR (non-meal -> meal)", np.mean(false_positive))
print("FNR (meal -> non-meal)", np.mean(true_negative))
print("Accuracy: ", np.mean(accuracy) )

TNR (non-meal -> non-meal) 0.7058823529411765
TPR (meal -> meal) 0.3130841121495327
FPR (non-meal -> meal) 0.29411764705882354
FNR (meal -> non-meal) 0.7058823529411765
Accuracy:  0.6019777503090235
